<a href="https://colab.research.google.com/github/rajeevsai12/SemanticsConversionAndDataGeneration/blob/main/CovidDeathsByCountryCsvToRdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# used the references of 'rdflib' from 'https://rdflib.readthedocs.io/en/stable/intro_to_parsing.html'

%%capture
!pip3 install rdflib sparqlwrapper pydotplus graphviz

import pandas as pd
from rdflib import Graph, Literal, Namespace, RDF, RDFS, URIRef, OWL
from rdflib.namespace import XSD

def create_rdf_graph(df, NS):
    # Create an RDF graph
    g = Graph()

    # Define RowId Class
    RowId = URIRef("http://example.org/RowId")
    g.add((RowId, RDF.type, OWL.Class))
   
    properties = list(df.columns.str.split())
    properties1 = list(df.columns)

    for i in range(len(properties)):
      properties[i] = ''.join(properties[i])

    for i in range(len(properties)):
      if '-' in properties[i]:
        properties[i] = properties[i].replace('-','')

    for p in properties:
        g.add((NS.p, RDF.type, OWL.DatatypeProperty))
        
    # Iterate over the rows in the DataFrame and create RDF triples for each row
    for _, row in df.iterrows():
        row_uri = URIRef(str(NS)+"row"+str(row['FIPS County Code']))
        g.add((row_uri, RDF.type, NS.RowId))
        for p in range(len(properties1)):
            g.add((row_uri, NS[properties[p]], Literal(row[properties1[p]], datatype=XSD.string)))
    return g

NS = Namespace("http://covide-19-deaths.org/")
df = pd.read_csv("Provisional_COVID-19_Death_Counts_in_the_United_States_by_County.csv")
g = create_rdf_graph(df, NS)
g.serialize(destination='Covid19CountryDeaths.rdf',format='xml')